In [1]:
import os
import csv
import pandas as pd
import glob
os.add_dll_directory(r'c:\Program Files\IBM\IBM DATA SERVER DRIVER\bin')
import ibm_db
pd.options.mode.chained_assignment = None  # default='warn'

def get_csv_paths(root_dir):
    paths = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.csv') and file.startswith('F'):
                paths.append(os.path.join(subdir, file))
    return paths

def read_csv_file(file_path):
    separators = ['\t', ' ']
    for sep in separators:
        try:
            with open(file_path, 'r', encoding='utf-16') as file:
                reader = csv.reader(file, delimiter=sep)
                data = list(reader)
                if len(data[0]) > 1:
                    return data
        except UnicodeError:
            continue
    return None

def process_csv_data(csv_data):
    df_list = []

    for row in csv_data:
        if len(row) == 4:
            df_list.append(pd.DataFrame({'Name1': [row[0]], 'Value1': [row[1]], 'Name2': [row[2]], 'Value2': [row[3]]}))
        elif len(row) == 2:
            df_list.append(pd.DataFrame({'Name1': [row[0]], 'Value1': [row[1]]}))

    if df_list:
        df = pd.concat(df_list, ignore_index=True)

        df1 = df[['Name1', 'Value1']].copy()
        df1.columns = ['Name', 'Value']

        df2 = df[['Name2', 'Value2']].dropna().copy()
        df2.columns = ['Name', 'Value']

        final = pd.concat([df1, df2])
        return final
    
    return None


def calculate_cpk(final, file_path):

    kyori = final[final['Name'].str.startswith('距離')]
    kyori['Value'] = kyori['Value'].astype(float)
    kyori['Value'] = kyori['Value'] *1000
    mean_value = kyori['Value'].mean()
    std_deviation = kyori['Value'].std()
    cpk = (13.5 - mean_value) / (3 * std_deviation)
    
    # return kyori['Value']
    return pd.DataFrame({
        'MATERIAL_ID': ['22KA662000' + file_path[-8:-4]],
        '平均値': [mean_value],
        '標準偏差': [std_deviation],
        'Cpk': [cpk]
    })


def main(root_dir):
    csv_paths = get_csv_paths(root_dir)
    df_list = []

    for path in csv_paths:
        csv_data = read_csv_file(path)
        if csv_data:
            final_df = process_csv_data(csv_data)
            final_df = calculate_cpk(final_df, path)
  
            if final_df is not None:
                df_list.append(final_df)

    target = pd.concat(df_list, ignore_index=True)
    # return final_df
    return target


# Main
root_dir = r'C:\Users\00220401626\Desktop\E1G0.6600'
target = main(root_dir)
target = target.round(2)

target

,MATERIAL_ID,平均値,標準偏差,Cpk
0,22KA6620004502,4.51,2.38,1.26
1,22KA6620004503,2.89,2.32,1.52
2,22KA6620004504,3.60,1.99,1.66
3,22KA6620004509,4.18,2.08,1.49
4,22KA6620004510,2.78,1.99,1.80
5,22KA6620004511,4.65,2.41,1.22
6,22KA6620004512,3.05,1.99,1.75
7,22KA6620004513,4.14,1.69,1.84
8,22KA6620004514,3.06,1.41,2.47
9,22KA6620004522,4.43,2.36,1.28


In [ ]:
server   = "10.143.16.244"
database = "AYB_APPL"
username = "IOT_DATA"
password = "asd23fgh"
conn = ibm_db.connect("DATABASE=" + database + ";HOSTNAME=" + server + ";PORT=50000;PROTOCOL=TCPIP;UID=" + username + ";PWD=" + password + ";", "", "")

if conn:
    print("Connected")
    
list_db = ['B31A', 'B28', 'B20L', 'B15', 'B21', 'B36A', 'B29']  


query_template = """
        SELECT DISTINCT
            *
        FROM
            IOT_DATA.{}_ID_PROCESS tb
        WHERE
        tb.MATERIAL_ID IN (
                '22KA6620004502',
                '22KA6620004503',
                '22KA6620004504',
                '22KA6620004509',
                '22KA6620004510',
                '22KA6620004511',
                '22KA6620004512',
                '22KA6620004513',
                '22KA6620004514',
                '22KA6620004522'
            );
    """

for db in list_db:

    query = query_template.format(db)
    stmt = ibm_db.exec_immediate(conn, query)

    result_list = []
    row = ibm_db.fetch_assoc(stmt)
    while row:
        result_list.append(row)
        row = ibm_db.fetch_assoc(stmt)

    df = pd.DataFrame(result_list)
    df = df.dropna(axis=1, how='all')
    columns_to_drop = ['UNIT_NUM', 'PARTS_NUM', 'RECIPE_CODE', 'PN', 'JN', 'OPENO', 'LOTNO', 'LOT_NUM']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    df['MATERIAL_ID'] = df['MATERIAL_ID'].str.strip()

    cols = df.columns.tolist()  
  
    cols.insert(0, cols.pop(cols.index('MATERIAL_ID')))  
    cols.insert(1, cols.pop(cols.index('UNIT_NAME'))) 
    cols.insert(2, cols.pop(cols.index('MEASURED_DT')))  
    cols.insert(3, cols.pop(cols.index('ST_TIME'))) 
    cols.insert(4, cols.pop(cols.index('END_TIME')))
    cols.insert(5, cols.pop(cols.index('EQP_ID')))

    df = df[cols]  # Reorder the DataFrame
 
    db_mapping = {
                'B31A': "B31A_デスミア",
                'B28': "B28_ABFラミ",
                'B20L': "B20L_PTNめっき",
                'B15': "B15_無電解銅めっき",
                'B21': "B21_レジスト剥離",
                'B36A': "B36A_回路形成",
                'B29': "B29_予備便化"
                }
    
    if db in db_mapping:
        mapped_db = db_mapping[db]


    root_path = r"c:\Users\00220401626\Desktop\22KA662X60R_OutPut"
    full_path = os.path.join(root_path, mapped_db)
    df.to_excel(full_path + '.xlsx', index=False)


In [2]:
folder_1 = r'c:\Users\00220401626\Desktop\22KA662X60R_OutPut\*.xlsx'  
excel_files = glob.glob(folder_1)
output_folder = r'c:\Users\00220401626\Desktop\240628' 

for file in excel_files:
    print(f'{file} started')
    data = pd.read_excel(file)

    df = pd.DataFrame(target['MATERIAL_ID'].unique(), columns=['MATERIAL_ID'])

    if 'LOG_NUM' in data.columns and 'UNIT_NAME' in data.columns:
        data['UNIT_NAME'] = data['UNIT_NAME'].astype(str) + '_' + data['LOG_NUM'].astype(str)

    data.drop(columns=['LOG_NUM'], inplace=True)
    unique_units = data['UNIT_NAME'].unique()
   
    for unit in unique_units: 
        print(unit)
       
        filtered_data = data[(data['UNIT_NAME'] == unit) & (data['MATERIAL_ID'].isin(target['MATERIAL_ID']))]
        df = pd.merge(df, filtered_data, on='MATERIAL_ID', how='left', suffixes=('', f'_{unit}'))

        df.dropna(axis=1, how='all', inplace=True)
        df = df.sort_values(by='ST_TIME')
        df = df.drop_duplicates('MATERIAL_ID', keep='first')
        df = df.sort_values(by='MATERIAL_ID')


    filename = os.path.basename(file).replace('.xlsx', '_processed.xlsx')
    output_path = os.path.join(output_folder, filename)

    df.to_excel(output_path, index=False)
    print(f'{file} finished')


c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B15_無電解銅めっき.xlsx started
水平無電解銅めっき_1
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B15_無電解銅めっき.xlsx finished
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B20L_PTNめっき.xlsx started
プラズマ処理_1
PTNめっき_1
PTNめっき_2
水平乾燥_1
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B20L_PTNめっき.xlsx finished
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B21_レジスト剥離.xlsx started
レジスト剥離_1
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B21_レジスト剥離.xlsx finished
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B28_ABFラミ.xlsx started
非接触クリーナー_1
ラミネーター_1
真空プレス_1
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B28_ABFラミ.xlsx finished
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B29_予備便化.xlsx started
連続オーブン_1
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B29_予備便化.xlsx finished
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B31A_デスミア.xlsx started
デスミア処理_1
デスミア処理_2
ビア底洗浄_1
ビア底洗浄_2
c:\Users\00220401626\Desktop\22KA662X60R_OutPut\B31A_デスミア.xlsx finished
c:\Users

In [3]:
folder_2 = r'c:\Users\00220401626\Desktop\240628\*.xlsx' 
excel_files = glob.glob(folder_2)

df = pd.DataFrame(target['MATERIAL_ID'].unique(), columns=['MATERIAL_ID'])

for file in excel_files:
 
    data = pd.read_excel(file)
    df = pd.merge(df, data, on='MATERIAL_ID', how='left', suffixes=('_left', f'_right'))

extra_columns = {
    "製品サイズ": ["27.7x24.0"] * 10,
    "シートサイズ": ["271.35x234.25"] * 10,
    "製品取り数/pnl": [352] * 10,
    "部品数/Pcs": [0] * 10,
    "部品搭載": ["なし(ダミーパターン加工)"] * 10,
    "ABF材料": ["GX92R"] * 10,
    "予備硬化条件": ["レシピ16"] * 10,
    "デスミア条件": ["レシピ1"] * 10,
    "シードエッチ条件": ["レシピ3"] * 10,
}


for col, values in extra_columns.items():
    df[col] = values
    
df.to_excel(r'c:\Users\00220401626\Desktop\test\final_1.xlsx', index=False)

C:\Users\00220401626\AppData\Local\Temp\ipykernel_19504\2965948536.py:9: FutureWarning: Passing 'suffixes' which cause duplicate columns {'A79_left', 'A53_left', 'A26_left', 'A60_left', 'A21_left', 'A47_left', 'A54_left', 'A4_left', 'A44_left', 'A69_left', 'A7_left', 'A19_left', 'A42_left', 'A75_left', 'A82_left', 'A39_left', 'A61_left', 'A81_left', 'A57_left', 'END_TIME_left', 'A50_left', 'A17_left', 'A11_left', 'A1_left', 'A40_left', 'A18_left', 'A83_left', 'A62_left', 'A27_left', 'MEASURED_DT_left', 'A48_left', 'A71_left', 'A2_left', 'A13_left', 'A87_left', 'A29_left', 'A86_left', 'A59_left', 'A34_left', 'A43_left', 'A20_left', 'A24_left', 'A16_left', 'A55_left', 'A58_left', 'A85_left', 'UNIT_NAME_left', 'A76_left', 'A12_left', 'A5_left', 'A73_left', 'A56_left', 'A66_left', 'A23_left', 'A35_left', 'A30_left', 'A77_left', 'A84_left', 'A3_left', 'A36_left', 'EQP_ID_left', 'A78_left', 'A65_left', 'A32_left', 'ST_TIME_left', 'A51_left', 'A9_left', 'A33_left', 'A8_left', 'A88_left', 'A6_

In [4]:
asv = pd.read_excel(r'c:\Users\00220401626\Desktop\VIASHIFT_KI_20240628.xlsx', sheet_name= '22KA662X60R ※水色背景は設備データ有')

asv = asv[asv['入力日'] <= '2024-01-25']
asv = asv[asv['処理種'].isin(['COMP', 'OPST'])]
asv['入力時'] = pd.to_datetime(asv['入力時'], format='%H:%M:%S').dt.time
asv['Datetime'] = pd.to_datetime(asv['入力日'].dt.strftime('%Y-%m-%d') + ' ' + asv['入力時'].astype(str))
asv = asv[['Datetime', '処理種', 'オペNo.元', '工程名' ]]

opn = asv['オペNo.元'].unique()

results = []

for i in opn:
    max_time = asv.loc[(asv['オペNo.元'] == i) & (asv['処理種'] == 'OPST'), 'Datetime'].max()
    start_time_row = asv[(asv['オペNo.元'] == i) & (asv['処理種'] == 'OPST') & (asv['Datetime'] == max_time)]

    if not start_time_row.empty:
        comp_time_row = asv[(asv['Datetime'] < start_time_row['Datetime'].values[0]) & (asv['処理種'] == 'COMP')].max()
        
        if not comp_time_row.empty:
            time_difference = start_time_row['Datetime'].values[0] - comp_time_row['Datetime']
            results.append({
                'オペNo.元': i,
                'Start Time': start_time_row['Datetime'].values[0],
                'Comp Time': comp_time_row['Datetime'],
                'Time Difference': time_difference
            })

result = pd.DataFrame(results)
result = result[['オペNo.元', 'Time Difference']]

for index, row in result.iterrows():
    result[row['オペNo.元']] = row['Time Difference'] 
 
result.drop(columns=['オペNo.元', 'Time Difference'], inplace=True)
result = result.iloc[:10]
result['MATERIAL_ID'] = target['MATERIAL_ID']
result

,E1G0.5500,E1G0.4000,E1G0.3600,E1G0.3500,E1G0.3000,E1D0.6000,E1D0.5000,E1D0.4000,E1D0.3000,E1D0.2000,...,A0H0.3500,A0H0.3000,A0H0.1000,A0C1.7100,A0C1.6500,A0C1.6000,A0C1.5000,A0C1.3000,A0C1.1000,MATERIAL_ID
0,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004502
1,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004503
2,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004504
3,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004509
4,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004510
5,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004511
6,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004512
7,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004513
8,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004514
9,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004522


In [19]:
#製品情報
extra_columns = {
    "製品サイズ": ["27.7x24.0"] * 10,
    "シートサイズ": ["271.35x234.25"] * 10,
    "製品取り数/pnl": [352] * 10,
    "部品数/Pcs": [0] * 10,
    "部品搭載": ["なし(ダミーパターン加工)"] * 10,
    "ABF材料": ["GX92R"] * 10,
    "予備硬化条件": ["レシピ16"] * 10,
    "デスミア条件": ["レシピ1"] * 10,
    "シードエッチ条件": ["レシピ3"] * 10,
}

#d "days" h:mm:ss
extra_df = pd.DataFrame(extra_columns)
extra_df['MATERIAL_ID']= pd.DataFrame(target['MATERIAL_ID'].unique(), columns=['MATERIAL_ID'])
main = pd.merge(extra_df, result, on='MATERIAL_ID', how='left')
main

,製品サイズ,シートサイズ,製品取り数/pnl,部品数/Pcs,部品搭載,ABF材料,予備硬化条件,デスミア条件,シードエッチ条件,MATERIAL_ID,...,A0H0.4000,A0H0.3500,A0H0.3000,A0H0.1000,A0C1.7100,A0C1.6500,A0C1.6000,A0C1.5000,A0C1.3000,A0C1.1000
0,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004502,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
1,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004503,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
2,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004504,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
3,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004509,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
4,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004510,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
5,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004511,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
6,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004512,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
7,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004513,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
8,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004514,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT
9,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004522,...,0 days 13:06:36,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT


In [23]:
folder = r'c:\Users\00220401626\Desktop\240628\*.xlsx' 
output_folder = r'c:\Users\00220401626\Desktop\test' 
excel_files = glob.glob(folder)

for file in excel_files:
    data = pd.read_excel(file)
    df = pd.merge(data, main, on='MATERIAL_ID', how='left')
    final = pd.merge(df, target, on='MATERIAL_ID', how='left')
    filename = os.path.basename(file)
    output_path = os.path.join(output_folder, filename)
    final.to_excel(output_path, index=False)
    print(output_path)

c:\Users\00220401626\Desktop\test\B15_無電解銅めっき_processed.xlsx
c:\Users\00220401626\Desktop\test\B20L_PTNめっき_processed.xlsx
c:\Users\00220401626\Desktop\test\B21_レジスト剥離_processed.xlsx
c:\Users\00220401626\Desktop\test\B28_ABFラミ_processed.xlsx
c:\Users\00220401626\Desktop\test\B29_予備便化_processed.xlsx
c:\Users\00220401626\Desktop\test\B31A_デスミア_processed.xlsx
c:\Users\00220401626\Desktop\test\B36A_回路形成_processed.xlsx


In [5]:
main = pd.merge(df, result, on='MATERIAL_ID', how='left')
final_df = pd.merge(main, target, on='MATERIAL_ID', how='left')
final_df

,MATERIAL_ID,UNIT_NAME_left,MEASURED_DT_left,ST_TIME_left,END_TIME_left,EQP_ID_left,A1_left,A2_left,A3_left,A4_left,...,A0H0.1000,A0C1.7100,A0C1.6500,A0C1.6000,A0C1.5000,A0C1.3000,A0C1.1000,平均値,標準偏差,Cpk
0,22KA6620004502,水平無電解銅めっき_1,2023-12-12 11:29:47,2023-12-12 10:58:04,2023-12-12 11:29:47,LEK10001,39,332,79.900002,80.099998,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.51,2.38,1.26
1,22KA6620004503,水平無電解銅めっき_1,2023-12-12 11:31:09,2023-12-12 10:59:26,2023-12-12 11:31:09,LEK10001,39,333,80.000000,79.900002,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,2.89,2.32,1.52
2,22KA6620004504,水平無電解銅めっき_1,2023-12-12 11:32:31,2023-12-12 11:00:48,2023-12-12 11:32:31,LEK10001,39,334,80.599998,79.699997,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,3.60,1.99,1.66
3,22KA6620004509,水平無電解銅めっき_1,2023-12-12 11:29:23,2023-12-12 11:07:36,2023-12-12 11:29:23,LEK10001,39,339,80.000000,79.699997,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.18,2.08,1.49
4,22KA6620004510,水平無電解銅めっき_1,2023-12-12 11:30:45,2023-12-12 11:08:58,2023-12-12 11:30:45,LEK10001,39,340,80.099998,80.099998,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,2.78,1.99,1.80
5,22KA6620004511,水平無電解銅めっき_1,2023-12-12 11:32:07,2023-12-12 11:10:20,2023-12-12 11:32:07,LEK10001,39,341,80.699997,79.400002,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.65,2.41,1.22
6,22KA6620004512,水平無電解銅めっき_1,2023-12-12 11:33:28,2023-12-12 11:11:42,2023-12-12 11:33:28,LEK10001,39,342,79.599998,79.699997,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,3.05,1.99,1.75
7,22KA6620004513,水平無電解銅めっき_1,2023-12-12 11:28:37,2023-12-12 11:13:04,2023-12-12 11:28:37,LEK10001,39,343,79.699997,80.000000,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.14,1.69,1.84
8,22KA6620004514,水平無電解銅めっき_1,2023-12-12 11:29:59,2023-12-12 11:14:25,2023-12-12 11:29:59,LEK10001,39,344,80.400002,79.900002,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,3.06,1.41,2.47
9,22KA6620004522,水平無電解銅めっき_1,2023-12-12 11:34:21,2023-12-12 11:25:19,2023-12-12 11:34:21,LEK10001,39,352,80.000000,79.800003,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.43,2.36,1.28


In [6]:
final_df.to_excel(r'c:\Users\00220401626\Desktop\test\final_1.xlsx', index=False)

In [11]:
import pandas as pd

final_df = pd.read_excel(r'c:\Users\00220401626\Desktop\test\final_1.xlsx')

list_of_columns = final_df.columns.tolist()  
duplicates = [column for column in list_of_columns if list_of_columns.count(column) > 1]

# print(list_of_columns)
print(duplicates)

[]


In [10]:
final_df.columns

Index(['MATERIAL_ID', 'UNIT_NAME_left', 'MEASURED_DT_left', 'ST_TIME_left',
       'END_TIME_left', 'EQP_ID_left', 'A1_left', 'A2_left', 'A3_left',
       'A4_left',
       ...
       'A0H0.1000', 'A0C1.7100', 'A0C1.6500', 'A0C1.6000', 'A0C1.5000',
       'A0C1.3000', 'A0C1.1000', '平均値', '標準偏差', 'Cpk'],
      dtype='object', length=1695)

In [ ]:
# "デスミア"            E1G0.5500
# "ABFラミ"             E1D0.5000, E1D0.2000
# "PTNめっき"           A0V0.7000
# "無電解銅めっき"       A0V0.1000
# "レジスト剥離"         A0V0.7250
# "回路形成"            A0V0.5000
# "予備便化"            E1D0.3000, E1D0.6000
# "ﾚｰｻﾞｰﾄﾞﾘﾙ(Front)"    A0H0.3000

# B31A : デスミア
# B28  : ABFラミ
# B20L : PTNめっき
# B15  : 無電解銅めっき
# B21  : レジスト剥離
# B36A : 回路形成
# B29  : 予備便化

In [ ]:
a = pd.read_excel(r'c:\Users\00220401626\Desktop\240628\B20L_PTNめっき_processed.xlsx')

In [ ]:
column_str = a['UNIT_NAME'][0]
column_str

In [ ]:
for i in a.columns:
    print(i)

In [ ]:
# Find the index where columns start with 'UNIT_NAME'
start_index = next((i for i, col in enumerate(a.columns) if col.startswith('UNIT_NAME')), len(a.columns))

# Get the columns to rename
columns_to_rename = a.columns[:start_index]

# Rename columns with suffix
a.rename(columns={col: col + column_str for col in columns_to_rename}, inplace=True)

